## 📊 Player Profiling

**Competitions:** TBC  
**Purpose:** TBC  
**Methods:** TBC  
**Author:** [Victoria Friss de Kereki](https://www.linkedin.com/in/victoria-friss-de-kereki/)  

---

**Notebook first written:** `13/02/2026`  
**Last updated:** `13/02/2026`  

> TBC


# Packages and configuration

In [12]:
from statsbombpy import sb
import pandas as pd
from mplsoccer import VerticalPitch,Pitch
from highlight_text import ax_text, fig_text
from matplotlib.colors import LinearSegmentedColormap
import matplotlib.pyplot as plt
import matplotlib.patheffects as path_effects
import seaborn as sns
import pprint
import numpy as np

pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)

# Core
from datetime import datetime, timedelta
import os

# Data manipulation
import numpy as np
import pandas as pd

# APIs & environment
import requests
from dotenv import load_dotenv
import time  # for delaying requests
from datetime import date

# Statistics
from scipy.stats import poisson

# Visualisation
import matplotlib.pyplot as plt
from matplotlib.colors import LinearSegmentedColormap
import dataframe_image as dfi # To download the final images

# Nicer printing of tables, no wrapping
pd.set_option("display.width", None)
pd.set_option("display.max_columns", None)
pd.set_option("display.expand_frame_repr", False)

# Load Competiton, Match, and Event Data from statsbombpy

In [3]:
# Call statsbombpy API to get all free competitions
free_comps = sb.competitions()

# Check Women's comps
free_comps = sb.competitions()
women_comps = free_comps[free_comps['competition_gender'] == 'female']
women_comps

,competition_id,season_id,country_name,competition_name,competition_gender,competition_youth,competition_international,season_name,match_updated,match_updated_360,match_available_360,match_available
25,37,90,England,FA Women's Super League,female,False,False,2020/2021,2025-04-23T14:16:46.924831,2021-06-13T16:17:31.694,None,2025-04-23T14:16:46.924831
26,37,42,England,FA Women's Super League,female,False,False,2019/2020,2024-02-12T15:05:34.211400,2021-06-13T16:17:31.694,None,2024-02-12T15:05:34.211400
27,37,4,England,FA Women's Super League,female,False,False,2018/2019,2024-08-07T17:22:40.334287,2021-06-13T16:17:31.694,None,2024-08-07T17:22:40.334287
63,49,3,United States of America,NWSL,female,False,False,2018,2024-12-15T12:31:48.035735,2021-06-13T16:17:31.694,None,2024-12-15T12:31:48.035735
71,53,315,Europe,UEFA Women's Euro,female,False,True,2025,2025-07-28T14:19:20.467348,2025-07-29T16:03:07.355174,2025-07-29T16:03:07.355174,2025-07-28T14:19:20.467348
72,53,106,Europe,UEFA Women's Euro,female,False,True,2022,2024-02-13T13:27:17.178263,2024-02-13T13:30:52.820588,2024-02-13T13:30:52.820588,2024-02-13T13:27:17.178263
73,72,107,International,Women's World Cup,female,False,True,2023,2025-07-14T10:07:06.620906,2025-07-14T10:10:27.224586,2025-07-14T10:10:27.224586,2025-07-14T10:07:06.620906
74,72,30,International,Women's World Cup,female,False,True,2019,2024-08-08T15:57:56.748740,2021-06-13T16:17:31.694,None,2024-08-08T15:57:56.748740


# FA Women's Super League

In [10]:
# List of season IDs
season_ids = [90, 42, 4]

# Fetch each season and combine into a single DataFrame
matches_list = [sb.matches(competition_id=37, season_id=s) for s in season_ids]
matches_england = pd.concat(matches_list, ignore_index=True)

matches_england.head(5)

,match_id,match_date,kick_off,competition,season,home_team,away_team,home_score,away_score,match_status,match_status_360,last_updated,last_updated_360,match_week,competition_stage,stadium,referee,home_managers,away_managers,data_version,shot_fidelity_version,xy_fidelity_version
0,3775648,2021-02-28,15:00:00.000,England - FA Women's Super League,2020/2021,Aston Villa,Arsenal WFC,0,4,available,scheduled,2022-11-29T22:20:07.837668,2021-06-13T16:17:31.694,11,Regular Season,Poundland Bescot Stadium,Lisa Benn,Marcus Bignot,Joseph Montemurro,1.1.0,2,2
1,3775609,2021-04-28,20:30:00.000,England - FA Women's Super League,2020/2021,Arsenal WFC,West Ham United LFC,2,0,available,scheduled,2023-02-24T20:39:46.594213,2021-06-13T16:17:31.694,13,Regular Season,Meadow Park,Emily Heaslip,Joseph Montemurro,Olli Harder,1.1.0,2,2
2,3775633,2021-02-06,13:30:00.000,England - FA Women's Super League,2020/2021,Aston Villa,Tottenham Hotspur Women,1,0,available,scheduled,2022-11-29T22:16:43.212186,2021-06-13T16:17:31.694,14,Regular Season,Poundland Bescot Stadium,Jane Simms,Marcus Bignot,Rehanne Skinner,1.1.0,2,2
3,3775570,2021-03-28,13:30:00.000,England - FA Women's Super League,2020/2021,Brighton & Hove Albion WFC,Everton LFC,0,5,available,scheduled,2021-04-13T21:46:33.264800,2021-06-13T16:17:31.694,18,Regular Season,Broadfield Stadium,Christina Hattersley,Hope Patricia Powell,Willie Kirk,1.1.0,2,2
4,3775581,2021-03-28,15:30:00.000,England - FA Women's Super League,2020/2021,Chelsea FCW,Aston Villa,2,0,available,scheduled,2023-02-24T20:38:29.557046,2021-06-13T16:17:31.694,18,Regular Season,Kingsmeadow,Sarah Garratt,Emma Hayes,Marcus Bignot,1.1.0,2,2


# api-football

In [13]:
# Load variables from API_KEY.env
load_dotenv("API_KEY.env")

API_KEY = os.getenv("API_FOOTBALL_API_KEY")

if API_KEY is None:
    raise ValueError("API_KEY not found. Check API_KEY.env")

print("API key loaded successfully")

API key loaded successfully


In [16]:
import http.client

conn = http.client.HTTPSConnection("v3.football.api-sports.io")

headers = {
    'x-apisports-key': API_KEY
    }

conn.request("GET", "/countries", headers=headers)

res = conn.getresponse()
data = res.read()

print(data.decode("utf-8"))


{"get":"countries","parameters":[],"errors":[],"results":171,"paging":{"current":1,"total":1},"response":[{"name":"Albania","code":"AL","flag":"https:\/\/media.api-sports.io\/flags\/al.svg"},{"name":"Algeria","code":"DZ","flag":"https:\/\/media.api-sports.io\/flags\/dz.svg"},{"name":"Andorra","code":"AD","flag":"https:\/\/media.api-sports.io\/flags\/ad.svg"},{"name":"Angola","code":"AO","flag":"https:\/\/media.api-sports.io\/flags\/ao.svg"},{"name":"Antigua-And-Barbuda","code":"AG","flag":"https:\/\/media.api-sports.io\/flags\/ag.svg"},{"name":"Argentina","code":"AR","flag":"https:\/\/media.api-sports.io\/flags\/ar.svg"},{"name":"Armenia","code":"AM","flag":"https:\/\/media.api-sports.io\/flags\/am.svg"},{"name":"Aruba","code":"AW","flag":"https:\/\/media.api-sports.io\/flags\/aw.svg"},{"name":"Australia","code":"AU","flag":"https:\/\/media.api-sports.io\/flags\/au.svg"},{"name":"Austria","code":"AT","flag":"https:\/\/media.api-sports.io\/flags\/at.svg"},{"name":"Azerbaijan","code":"AZ

In [22]:
countries_df = pd.DataFrame(json.loads(data.decode("utf-8"))["response"])
countries_df

,name,code,flag
0,Albania,AL,https://media.api-sports.io/flags/al.svg
1,Algeria,DZ,https://media.api-sports.io/flags/dz.svg
2,Andorra,AD,https://media.api-sports.io/flags/ad.svg
3,Angola,AO,https://media.api-sports.io/flags/ao.svg
4,Antigua-And-Barbuda,AG,https://media.api-sports.io/flags/ag.svg
...,...,...,...
166,Wales,GB-WLS,https://media.api-sports.io/flags/gb-wls.svg
167,World,None,None
168,Yemen,YE,https://media.api-sports.io/flags/ye.svg
169,Zambia,ZM,https://media.api-sports.io/flags/zm.svg


In [32]:
import http.client
import json
import pandas as pd

conn = http.client.HTTPSConnection("v3.football.api-sports.io")

headers = {
    'x-apisports-key': API_KEY
}

conn.request("GET", "/leagues", headers=headers)
res = conn.getresponse()
data = res.read()

# Parse JSON
json_data = json.loads(data.decode("utf-8"))

# Convert to DataFrame
leagues_df = pd.json_normalize(json_data["response"])

In [33]:
import pandas as pd

# explode seasons list → one row per season
leagues_seasons = leagues_df.explode("seasons").reset_index(drop=True)

# normalize season dict into columns
season_cols = pd.json_normalize(leagues_seasons["seasons"])

# combine with league info
leagues_seasons = pd.concat(
    [leagues_seasons.drop(columns=["seasons"]), season_cols],
    axis=1
)

leagues_seasons

,league.id,league.name,league.type,league.logo,country.name,country.code,country.flag,year,start,end,current,coverage.fixtures.events,coverage.fixtures.lineups,coverage.fixtures.statistics_fixtures,coverage.fixtures.statistics_players,coverage.standings,coverage.players,coverage.top_scorers,coverage.top_assists,coverage.top_cards,coverage.injuries,coverage.predictions,coverage.odds
0,4,Euro Championship,Cup,https://media.api-sports.io/football/leagues/4...,World,None,None,2008,2008-06-07,2008-06-29,False,True,True,False,False,False,True,True,True,True,False,True,False
1,4,Euro Championship,Cup,https://media.api-sports.io/football/leagues/4...,World,None,None,2012,2012-06-08,2012-07-01,False,True,True,False,False,False,True,True,True,True,False,True,False
2,4,Euro Championship,Cup,https://media.api-sports.io/football/leagues/4...,World,None,None,2016,2016-06-10,2016-07-10,False,True,True,True,True,True,True,True,True,True,False,True,False
3,4,Euro Championship,Cup,https://media.api-sports.io/football/leagues/4...,World,None,None,2020,2019-03-21,2021-07-11,False,True,True,True,True,True,True,True,True,True,False,True,False
4,4,Euro Championship,Cup,https://media.api-sports.io/football/leagues/4...,World,None,None,2024,2024-06-14,2024-07-14,True,True,True,True,True,True,True,True,True,True,False,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8031,1214,Ofc Pro League,Cup,https://media.api-sports.io/football/leagues/1...,World,None,None,2026,2026-01-17,2026-04-18,True,True,True,False,False,True,True,True,True,True,False,True,True
8032,1215,Obos Supercup,Cup,https://media.api-sports.io/football/leagues/1...,Norway,NO,https://media.api-sports.io/flags/no.svg,2026,2026-01-23,2026-03-07,True,True,False,False,False,False,True,True,True,True,False,True,True
8033,1216,Serie Rio De La Plata,Cup,https://media.api-sports.io/football/leagues/1...,World,None,None,2026,2026-01-11,2026-01-27,True,True,True,False,False,False,False,False,False,False,False,True,False
8034,1217,Fifa Women Champions Cup,Cup,https://media.api-sports.io/football/leagues/1...,World,None,None,2026,2025-10-08,2026-02-01,True,True,True,False,False,False,False,False,False,False,False,True,False


In [34]:
uruguay_leagues = leagues_seasons[
    leagues_seasons["country.name"] == "Uruguay"
]

uruguay_leagues

,league.id,league.name,league.type,league.logo,country.name,country.code,country.flag,year,start,end,current,coverage.fixtures.events,coverage.fixtures.lineups,coverage.fixtures.statistics_fixtures,coverage.fixtures.statistics_players,coverage.standings,coverage.players,coverage.top_scorers,coverage.top_assists,coverage.top_cards,coverage.injuries,coverage.predictions,coverage.odds
567,269,Segunda División,League,https://media.api-sports.io/football/leagues/2...,Uruguay,UY,https://media.api-sports.io/flags/uy.svg,2011,2011-10-01,2012-06-30,False,False,False,False,False,True,False,False,False,False,False,True,False
568,269,Segunda División,League,https://media.api-sports.io/football/leagues/2...,Uruguay,UY,https://media.api-sports.io/flags/uy.svg,2012,2012-10-13,2013-07-20,False,False,False,False,False,True,True,True,True,True,False,True,False
569,269,Segunda División,League,https://media.api-sports.io/football/leagues/2...,Uruguay,UY,https://media.api-sports.io/flags/uy.svg,2013,2013-10-12,2014-07-06,False,True,True,False,False,True,True,True,True,True,False,True,False
570,269,Segunda División,League,https://media.api-sports.io/football/leagues/2...,Uruguay,UY,https://media.api-sports.io/flags/uy.svg,2014,2014-09-13,2015-06-13,False,True,True,False,False,True,True,True,True,True,False,True,False
571,269,Segunda División,League,https://media.api-sports.io/football/leagues/2...,Uruguay,UY,https://media.api-sports.io/flags/uy.svg,2015,2015-10-24,2016-06-11,False,True,True,False,False,True,True,True,True,True,False,True,False
572,269,Segunda División,League,https://media.api-sports.io/football/leagues/2...,Uruguay,UY,https://media.api-sports.io/flags/uy.svg,2016,2016-09-10,2016-12-10,False,True,True,False,False,True,True,True,True,True,False,True,False
573,269,Segunda División,League,https://media.api-sports.io/football/leagues/2...,Uruguay,UY,https://media.api-sports.io/flags/uy.svg,2017,2017-04-15,2017-12-16,False,True,True,False,False,True,True,True,True,True,False,True,False
574,269,Segunda División,League,https://media.api-sports.io/football/leagues/2...,Uruguay,UY,https://media.api-sports.io/flags/uy.svg,2018,2018-03-03,2018-10-27,False,True,True,False,False,True,True,True,True,True,False,True,False
575,269,Segunda División,League,https://media.api-sports.io/football/leagues/2...,Uruguay,UY,https://media.api-sports.io/flags/uy.svg,2019,2019-05-04,2019-12-14,False,True,True,False,False,True,True,True,True,True,False,True,False
576,269,Segunda División,League,https://media.api-sports.io/football/leagues/2...,Uruguay,UY,https://media.api-sports.io/flags/uy.svg,2020,2020-08-12,2021-01-29,False,True,True,False,False,True,True,True,True,True,False,True,False
